In [1]:
#利用seurat对象的metadata(已经加入了barcode，umap坐标)，countsMartrix,genenameDataframe,harmony/pca降为坐标，构建scanpy对象（adata）
#包加载区#
import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd

In [24]:
#环境设置区#
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=300, fontsize=10, dpi_save=600)

#路径设置
inputfold='../../output/01HC-DLE-SLE_blood-result/test/'
pattern_genelist=["IL4R","AIM2","MZB1","IGKV3-20","IGHV3-30","CD14","IGHV3-23","KCNQ5","CD5","SPN","CD27","CD19","CD70"]

-----
anndata     0.10.9
scanpy      1.10.2
-----
Cython                      3.0.11
PIL                         10.4.0
anyio                       NA
arrow                       1.3.0
asttokens                   NA
attr                        24.2.0
attrs                       24.2.0
babel                       2.14.0
brotli                      1.1.0
certifi                     2024.08.30
cffi                        1.17.1
charset_normalizer          3.3.2
colorama                    0.4.6
comm                        0.2.2
cycler                      0.12.1
cython                      3.0.11
cython_runtime              NA
dateutil                    2.9.0
debugpy                     1.8.5
decorator                   5.1.1
defusedxml                  0.7.1
executing                   2.1.0
fastjsonschema              NA
fqdn                        NA
h5py                        3.11.0
idna                        3.8
igraph                      0.11.6
ipykernel                   6.29.5

In [3]:
#在python中构建adata
#因为文件比较多，如多一个步骤一个步骤的修改很容易出错，所以我们把这个过程封装为一个函数
#函数区#
def seurat_to_adata(counts,#R中导出的counts.mtx文件
                   meta,#R中导出的metadata.csv文件
                   gene_name,#R中导出的gene_names.csv文件
                   pca,#R中导出的pca.csv文件 or harmony.csv文件
                   reduction1,#TSNE数据为TSNE_1，UMAP数据为UMAP_1
                   reduction2):#TSNE数据为TSNE_2，UMAP数据为UMAP_2
    # 加载表达矩阵
    X = io.mmread(counts)
    # 创建anndata
    adata = anndata.AnnData(X=X.transpose().tocsr())
    cell_meta = pd.read_csv(meta)#metadata
    with open(gene_name, 'r') as f:gene_names = f.read().splitlines()
    adata.obs = cell_meta
    adata.obs.index = adata.obs['barcode']
    adata.var.index = gene_names
    pca = pd.read_csv(pca)
    pca.index = adata.obs.index
    adata.obsm['X_pca'] = pca.to_numpy()
    adata.obsm['X_tsne'] = np.vstack((adata.obs[reduction1].to_numpy(), adata.obs[reduction2].to_numpy())).T
    return adata

In [1]:
#正式pipline区
sce_test=seurat_to_adata(counts=inputfold+'counts.mtx',
                        meta=inputfold+'metadata.csv',
                        gene_name=inputfold+'gene_names.csv',
                        pca=inputfold+'harmony.csv',
                        reduction1='UMAP_1',reduction2='UMAP_2')
#看一下重构的数据是否正常
sce_test

#看一下聚类图和marker基因气泡图，发现是可以的，没问题。这个对象就可以在python使用scanpy进行任何你需要的分析了
sc.pl.tsne(sce_test, color=['manual_L2'], frameon=False)#这里的tsne坐标根据你存起来的坐标而定
sc.pl.DotPlot(sce_test,
              pattern_genelist, 
              log = True, 
              groupby='manual_L2').style(cmap='PRGn',
                                         dot_edge_color='black',
                                         dot_edge_lw=1).swap_axes(False).show()
#保存构建好的数据
sce_test.write(inputfold+'sce_test.h5ad')
#可以进行scenic分析

NameError: name 'seurat_to_adata' is not defined